# Import libraries and helpers

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import sys

In [ ]:
sys.path.append('./helpers_models/')

In [ ]:
sys.path.append('./data_visualization_and_augmentations/')

In [ ]:
sys.path.append('../torch_videovision/')

In [ ]:
sys.path.append('../video-classification/ResNetCRNN/')

In [ ]:
sys.path.append('./important_csvs/')

In [ ]:
from helpers_lstm import *

# Load training and validation sets

In [ ]:
tensor_transform = get_tensor_transform('ImageNet')

In [ ]:
train_transform = get_video_transform(2)
valid_transform = get_video_transform(0)

In [ ]:
df = pd.read_csv('./important_csvs/events_with_number_of_frames_stratified.csv')
df = get_df(df, 16, False)

In [ ]:
class_image_paths, end_idx = get_indices(df)

In [ ]:
train_loader = get_loader(16, 4, end_idx, class_image_paths, train_transform, tensor_transform)

In [ ]:
# show_batch(train_loader,4)

In [ ]:
df = pd.read_csv('./important_csvs/events_with_number_of_frames_stratified.csv')
df = get_df(df, 16, True)

In [ ]:
class_image_paths, end_idx = get_indices(df)

In [ ]:
valid_loader = get_loader(16, 4, end_idx, class_image_paths, valid_transform, tensor_transform)

# Load Model, change head, freeze body

In [ ]:
device = torch.device('cuda')

In [ ]:
cnn_encoder = ResCNNEncoder().to(device)
adaptive_pool = AdaptiveConcatPool2d()
cnn_encoder.resnet[8] = adaptive_pool

In [ ]:
for param in cnn_encoder.parameters():
    param.requires_grad = False
for param in cnn_encoder.resnet[8].parameters():
    param.requires_grad = True
for param in cnn_encoder.headbn1.parameters():
    param.requires_grad = True
for param in cnn_encoder.fc1.parameters():
    param.requires_grad = True

In [ ]:
rnn_decoder = DecoderRNNattention(batch_size=48).to(device)
for param in rnn_decoder.parameters():
    param.requires_grad = True

In [ ]:
crnn_params, cnn_encoder, rnn_decoder = parallelize_model(cnn_encoder, rnn_decoder)

## Torchsummary

In [ ]:
summary(cnn_encoder.module, (16,3,576,704))

In [ ]:
rnn_decoder.module

In [ ]:
summary(rnn_decoder.module, (16,512))

In [ ]:
h0, c0 = torch.from_numpy(np.zeros((2, 2, 512))).float(), torch.from_numpy(np.zeros((1, 64, 5))).float()

In [ ]:
h0.shape